In [ ]:
from flask import Flask,make_response,request
from flask_cors import CORS
from flask import jsonify
import math
import pandas as pd
app = Flask(__name__)
CORS(app)

##Get Total Balance of all debts list
def getTotalBal(debtdict):
    b=0
    for key, value in debtdict.items():
        if type(value) is dict:
            b+=value['balance']
    debtdict['totalbal']=b
  
##allocateExtraMoney to first loan with balance greater than zero
def allocateExtraMoney(debtdict, extraMoney):
   if debtdict['payminimum']==False:
       for key, value in debtdict.items():
            if type(value) is dict:
                if value['balance']>0.1:
                    value['minimumdue']+=extraMoney
                    break

def calculateMonthlyInterest(debt) :
	r = debt['interestrate'] / 100
	t = 1/12
	n = 365
	principal = debt['balance']
	newAmount =  principal * pow(1 + r/n , n * t)

	interestForMonth = math.ceil((newAmount - principal)*100)/100

	return interestForMonth
          
##Adds monthly interest to debt's balance, list's overall balance, and overall interest paid
def addMonthlyInterest(debtdict, debt,splitupdict):
      
    monthlyInterest = calculateMonthlyInterest(debt)
    splitupdict['interest'].append(monthlyInterest)
    debt['balance'] += monthlyInterest
    splitupdict['monthlybalance'].append(debt['balance'])
    debtdict['totalbal'] += monthlyInterest
    debt['totalinterestpaid'] += monthlyInterest

##Records payment in individual debt, and debt list. Increments months of payment for individual debt and debt list
def recordPayment(debtdict, debt, payment,splitupdict):
    debt['totalpaid'] += payment
    debt['months'] += 1
    splitupdict['monthlydue'].append(payment)
    splitupdict['month'].append(debt['months']) 
    debtdict['months'] = debt['months']
    debtdict['totalpaid']  += payment
    
def allocateSurplusPayment(debtdict, surplus):
    if debtdict['payminimum']==False:
    	if debtdict['totalbal'] > 0 :
            for key, value in debtdict.items():
                if type(value) is dict:
                    if value['balance']>0.1:
                        if value['balance']-surplus>0:
                            value['balance']-=surplus
                            #splitupdict['monthlybalance'][-1]=value['balance']
                            debtdict['totalbal']-=surplus
                            debtdict['totalpaid']+=surplus
                            value['totalpaid']+=surplus
                            break
                        else:
                            remainingSurplus = surplus -value['balance']
                            debtdict['totalbal']-=value['balance']
                            debtdict['totalpaid']+=value['balance']
                            value['totalpaid']+=value['balance']
                            value['balance']=0
                            #splitupdict['monthlybalance'][-1]=value['balance']
                            allocateExtraMoney(debtdict,value['minimumdue'])
                            allocateSurplusPayment(debtdict,remainingSurplus)
                            break
                                 
##Makes a monthly payment for a specific debt in the list. If debt is paid off, it calls functions to handle surplus
##and extra money from monthly payment
def makeMonthlyPayment(debtdict, debt,splitupdict):  
    payment = debt['minimumdue']
    isFinalPayment = False
    if (debt['balance']- payment < 0.1):
        isFinalPayment = True
        payment = debt['balance']
        surplus =  debt['minimumdue'] - payment
	
    debt['balance'] = math.ceil((debt['balance'] - payment)*100)/100
    debtdict['totalbal'] = math.ceil((debtdict['totalbal']  - payment)*100)/100
    splitupdict['monthlybalance'][-1]=debt['balance']
    recordPayment(debtdict, debt, payment,splitupdict)
    
    if isFinalPayment :
        allocateExtraMoney(debtdict,  debt['minimumdue'])
        allocateSurplusPayment(debtdict, surplus)
        isFinalPayment = False
	
##Iterates through all non zero debts, while the debt list total balance is
## greater than zero it calls functions to add monthly interest and make monthly payments.
## Each interation represents a month.        
def iterateMonths(debtdict,splitupdict):
    months=0
    while(debtdict['totalbal'] - 0.5 > 0 and months < 1200):  
        for key, value in debtdict.items():     
            if type(value) is dict:
                if value['balance']>0.1:  
                        addMonthlyInterest(debtdict,value,splitupdict[key])
                        makeMonthlyPayment(debtdict, value,splitupdict[key])
        months+=1
                

def snowball(debtslist,extraMoney):
##Sort based on interest for avalanche list
    debtslistsnowballdict={}
    debtlistsnowballdict={}
    monthsplitupdict_snowball={}
    count=0
   ##Sort based on balance for snowball list
    debtslist.sort(key = lambda x: x[1]) 
    debtslistsnowball=debtslist.copy()
    print("Debt list snowball:")
    print(debtslistsnowball)
    print("--------------------------")    
    ##Create snowball dictionary
    for debtinfo in debtslistsnowball:
        debtlistsnowballdict['debt']=debtinfo[0]
        debtlistsnowballdict['minimumdue']=debtinfo[1]
        debtlistsnowballdict['interestrate']=debtinfo[2]
        debtlistsnowballdict['balance']=debtinfo[3]
        debtlistsnowballdict['totalpaid']=debtinfo[4]
        debtlistsnowballdict['months']=debtinfo[5]
        debtlistsnowballdict['totalinterestpaid']=debtinfo[6]
        debtslistsnowballdict.update({'debt'+str(count):debtlistsnowballdict})
        debtlistsnowballdict={}
        count+=1
    for key in debtslistsnowballdict:
        monthsplitupdict_snowball[key]={'month':[],'interest':[],'monthlybalance':[],'monthlydue':[]}
  
    debtslistsnowballdict['totalpaid']=0
    debtlistsnowballdict['totalbal']=0
    debtslistsnowballdict['months']=0
    debtslistsnowballdict['payminimum']=False
    getTotalBal(debtslistsnowballdict)
    allocateExtraMoney(debtslistsnowballdict, extraMoney)
    iterateMonths(debtslistsnowballdict,monthsplitupdict_snowball)
    #print(debtslistsnowball)
  
    return debtslistsnowballdict,monthsplitupdict_snowball


def avalanche(debtslist,extraMoney):  
    debtslistavalanchedict={}
    debtlistavalanchedict={}
    monthsplitupdict_avalanche={}
    debtslist.sort(key = lambda x: x[2],reverse=True) 
    debtslistavalanche=debtslist.copy()
    print("Debt list avalanche:")
    print(debtslistavalanche)
    print("--------------------------")
    
    ##Create avalanche dictionary
    count=0
    for debtinfo in debtslistavalanche:
        debtlistavalanchedict['debt']=debtinfo[0]
        debtlistavalanchedict['minimumdue']=debtinfo[1]
        debtlistavalanchedict['interestrate']=debtinfo[2]
        debtlistavalanchedict['balance']=debtinfo[3]
        debtlistavalanchedict['totalpaid']=debtinfo[4]
        debtlistavalanchedict['months']=debtinfo[5]
        debtlistavalanchedict['totalinterestpaid']=debtinfo[6]
        debtslistavalanchedict.update({'debt'+str(count):debtlistavalanchedict})
        debtlistavalanchedict={}
        count+=1
    for key in debtslistavalanchedict:
        monthsplitupdict_avalanche[key]={'month':[],'interest':[],'monthlybalance':[],'monthlydue':[]}
    
    debtslistavalanchedict['totalpaid']=0
    debtslistavalanchedict['totalbal']=0
    debtslistavalanchedict['months']=0
    debtslistavalanchedict['payminimum']=False
    getTotalBal(debtslistavalanchedict)
    allocateExtraMoney(debtslistavalanchedict, extraMoney)
    iterateMonths(debtslistavalanchedict,monthsplitupdict_avalanche)
    return debtslistavalanchedict,monthsplitupdict_avalanche

def normal(debtslist,minimumdue):
    debtslistnormaldict={}
    debtlistnormaldict={}
    monthsplitupdict_normal={}
    print("Debt list normal:")
    print(debtslist)
    print("--------------------------")
    ##Create normal dictionary
    count=0
    for debtinfo in debtslist:
        debtlistnormaldict['debt']=debtinfo[0]
        debtlistnormaldict['minimumdue']=debtinfo[1]
        debtlistnormaldict['interestrate']=debtinfo[2]
        debtlistnormaldict['balance']=debtinfo[3]
        debtlistnormaldict['totalpaid']=debtinfo[4]
        debtlistnormaldict['months']=debtinfo[5]
        debtlistnormaldict['totalinterestpaid']=debtinfo[6]
        debtslistnormaldict.update({'debt'+str(count):debtlistnormaldict})
        debtlistnormaldict={}
        count+=1
    j=0
    for key in debtslistnormaldict:
        monthsplitupdict_normal[key]={'month':[],'interest':[],'monthlybalance':[],'minimumdue':minimumdue[j],'monthlydue':[]}
        j+=1
        
    debtslistnormaldict['totalpaid']=0
    debtslistnormaldict['totalbal']=0
    debtslistnormaldict['months']=0
    debtslistnormaldict['payminimum']=True
    getTotalBal(debtslistnormaldict)
    allocateExtraMoney(debtslistnormaldict, 0)
    iterateMonths(debtslistnormaldict,monthsplitupdict_normal)
    return debtslistnormaldict,monthsplitupdict_normal

def converttocsv(dict_msplit):
    df=pd.DataFrame()
    for key, value in dict_msplit.items():     
            if type(value) is dict:
                df1=pd.DataFrame.from_dict(dict_msplit[key])
                df=pd.concat([df,df1],axis=1)
    return df

def savinginterest(snowball_out,avalanche_out):
    if snowball_out['totalpaid']<avalanche_out['totalpaid']:
        return "Snowball technique",snowball_out
    else:
        return "Avalanche technique",avalanche_out

def feasiblityinloans(snowball_out,avalanche_out): 
    if snowball_out['months']<avalanche_out['months']:
        return "Snowball technique",snowball_out
    elif snowball_out['months']>avalanche_out['months'] :
        return "Avalanche technique",avalanche_out  
    else:
        if snowball_out['totalpaid']<avalanche_out['totalpaid']:
            return "Snowball technique",snowball_out
        else:
            return "Avalanche technique",avalanche_out
        
def debtfreeandinvestment(snowball_out,avalanche_out,snowball_msplit,avalanche_msplit,rateofinterest,duration,monthlyleftover,snb,avl):  
        cashflow_sb=0
        diff1=0
        diff2=0
        csb=[]
        cal=[]
        cal1=[]
        cashflow_av1=0
        csb1=[]
        cashflow_sb1=0
        if snowball_out['totalpaid']<=avalanche_out['totalpaid']:
           diff1=avalanche_out['totalpaid']-snowball_out['totalpaid']
        else:
            diff2=snowball_out['totalpaid']-avalanche_out['totalpaid']
        for key,value in snowball_msplit.items():
             if type(value) is dict:
                 if value['month']==snowball_out['months']:
                     cashflow_sb+=value['monthlybalance'][-2]+value['interest'][-1]
        if duration< snowball_out['months']:
            for i in range(duration):
                csb1.append(0)
        else:
            for i in range(snowball_out['months']-1):
                csb.append(0)
            cashflow_sb=monthlyleftover-cashflow_sb+diff1
            cashflow_sb=cashflow_sb*((1+(rateofinterest/1200))**12)
            csb.append(cashflow_sb)
            ctemp1=cashflow_sb
            cashflow_sb+=monthlyleftover*((1+(rateofinterest/1200))**(12*(duration-snowball_out['months'])))
            for i in range(duration-snowball_out['months']):
                ctemp1+=monthlyleftover*((1+(rateofinterest/1200))**12)
                csb.append(ctemp1)
        cashflow_av=0
        for key,value in avalanche_msplit.items():
                 if type(value) is dict:
                     if value['month']==avalanche_out['months']:
                         cashflow_av+=value['monthlybalance'][-2]+value['interest'][-1]
        if duration< avalanche_out['months']:
            for i in range(duration):
                cal1.append(0)
        else:
            for i in range(avalanche_out['months']-1):
                cal.append(0)
            cashflow_av=monthlyleftover-cashflow_av+diff2
            cashflow_av=cashflow_av*((1+(rateofinterest/1200))**12)
            cal.append(cashflow_av)
            ctemp2=cashflow_av
            cashflow_av+=monthlyleftover*((1+(rateofinterest/1200))**(12*(duration-avalanche_out['months'])))
            for i in range(duration-avalanche_out['months']):
                ctemp2+=monthlyleftover*((1+(rateofinterest/1200))**12)
                cal.append(ctemp2)
        if csb1!=[] and cal1!=[]:
            if snowball_out['totalpaid']<=avalanche_out['totalpaid']:
                print("snowball is better  for only paying debt")
                return cashflow_sb1,snowball_out,csb1,"snowball technique"
            else:
                 print("avalanche is better  for only paying debt")
                 return cashflow_av1,avalanche_out,cal1,"avalanche technique"
                 
        if cashflow_sb>=cashflow_av:
            print("snowball is better")
            return cashflow_sb,snowball_out,csb,"snowball technique followed by investment"
        else:
            print("avalanche is better")
            return cashflow_av,avalanche_out,cal,"avalanche technique followed by investment"
   
def debtminandparallelinvest(normal_out,normal_msplit,rateofinterest,duration,monthlyleftover):
    _max=0
    cnm=[]
    cnm1=[]
    cashflow1=0
    monthlyleftover_temp=monthlyleftover
    for key,value in normal_msplit.items():
             if type(value) is dict:
                 if _max<=len(value['month'])-1:
                     _max=len(value['month'])-1
    i=0
    clm=[]
    while i<=_max:
        monthlyleftover_temp=monthlyleftover
        for key,value in normal_msplit.items():
             if type(value) is dict:
                 if i<len(value['month']):                    
                     if value['monthlybalance'][i]>=value['minimumdue']:
                         monthlyleftover_temp-=value['minimumdue']
                     elif value['monthlybalance'][i]==0.0 :
                         monthlyleftover_temp-=value['monthlybalance'][i-1]+value['interest'][i]
        #print(monthlyleftover_temp)
        i+=1
        clm.append(monthlyleftover_temp)
    #print(clm)
    if duration<normal_out['months']:
        clm=clm[:duration]
        for i in clm:
            cashflow1+=i*((1+(rateofinterest/1200))**12)
            cnm1.append(cashflow1)
        return cashflow1,normal_out,cnm1,"Minimum due payment"
        
    cashflow=0
    for i in clm:
        cashflow+=i*((1+(rateofinterest/1200))**12)  
        cnm.append(cashflow)
    ctemp=cashflow
    cashflow+=monthlyleftover*((1+(rateofinterest/1200))**(12*(duration-_max)))
    for i in range(duration-_max-1):
            ctemp+=monthlyleftover*((1+(rateofinterest/1200))**12)
            cnm.append(ctemp)    
    return cashflow,normal_out,cnm,"Minimum due payment followed by  parallel investment"

def debtsaandparallelinvest(snb,avl,rateofinterest,duration,monthlyleftover):
    sb=[]
    csb=[]
    cal=[]
    sb1=[]
    for i in range(0,10):
        i=0.1*i
        csb=[]
        csb1=[]
        cashflow_sb=0
        cashflow_sb1=0
        #cashflow_sb1=0
        sbdict,sbmdict=snowball(snb,i*monthlyleftover)
        for key,value in sbmdict.items():
            if type(value) is dict:
                 if value['month']==sbdict['months']:
                     cashflow_sb+=value['monthlybalance'][-2]+value['interest'][-1]
        
        ctemp1=0
        diff=monthlyleftover-(i*monthlyleftover)
        cashflow_sb1+=diff*((1+(rateofinterest/1200))**(12*(sbdict['months'])))
        if duration<sbdict['months']:
            for i in range(duration):
                 ctemp1+=diff*((1+(rateofinterest/1200))**12)
                 csb1.append(ctemp1)
            sb1.append([i*monthlyleftover,cashflow_sb1,sbmdict,sbdict,csb1]) 
        else:
            ctemp1=0
            cashflow_sb+=diff*((1+(rateofinterest/1200))**(12*(sbdict['months'])))
            for i in range(sbdict['months']-1):
                ctemp1+=diff*((1+(rateofinterest/1200))**12)
                csb.append(ctemp1)
            cashflow_sb=(monthlyleftover-cashflow_sb)*((1+(rateofinterest/1200))**12)
            csb.append(cashflow_sb)
            ctemp1=cashflow_sb
            cashflow_sb+=monthlyleftover*((1+(rateofinterest/1200))**(12*(duration-sbdict['months'])))
            for i in range(duration-sbdict['months']):
                ctemp1+=monthlyleftover*((1+(rateofinterest/1200))**12)
                csb.append(ctemp1)
            sb.append([i*monthlyleftover,cashflow_sb,sbmdict,sbdict,csb])   
    av=[]
    av1=[]
    for i in range(0,10): 
        i=0.1*i
        cal=[]
        cal1=[]
        cashflow_av=0
        cashflow_av1=0
        #cashflow_av1=0
        avdict,avldict=avalanche(avl,i*monthlyleftover)     
        for key,value in avldict.items():
            if type(value) is dict:
                 if value['month']==avdict['months']:
                     cashflow_av+=value['monthlybalance'][-2]+value['interest'][-1]
        diff=monthlyleftover-(i*monthlyleftover)
        cashflow_av1+=diff*((1+(rateofinterest/1200))**(12*(duration)))
        ctemp2=0
        if duration<avdict['months']:
            for i in range(duration):
                 ctemp2+=diff*((1+(rateofinterest/1200))**12)
                 cal1.append(ctemp2)
            av1.append([i*monthlyleftover,cashflow_av1,avldict,avdict,cal1]) 
        else:
            ctemp2=0
            for i in range(avdict['months']-1):
                ctemp2+=diff*(1+(rateofinterest/1200))*12
                cal.append(ctemp2)
            cashflow_av+=diff*((1+(rateofinterest/1200))**(12*(avdict['months'])))
            cashflow_av+=(monthlyleftover-cashflow_av)*((1+(rateofinterest/1200))**12)
            csb.append(cashflow_av)
            cashflow_av+=monthlyleftover*((1+(rateofinterest/1200))**(12*(duration-avdict['months'])))
            for i in range(duration-avdict['months']):
                ctemp2+=monthlyleftover*((1+(rateofinterest/1200))**12)
                cal.append(ctemp2)
            av.append([i*monthlyleftover,cashflow_av,avldict,avdict,cal])  
    sbmax=0
    avmax=0
    c1=[]
    a1=[]
    snowmaxdict={}
    avmaxdixt={}
    if sb!=[] or av!=[]:
        for i in sb:
            if i[1]>sbmax:
                sbmax=i[1]
                snowmmax=i[2]
                snowmaxdict=i[3]
                c1=i[4]
        for i in av:
            if i[1]>avmax:
                avmax=i[1]
                avmmax=i[2]
                avmaxdixt=i[3]
                a1=i[4]
        if sbmax>=avmax:
            print("snow ball is better for doing debts with debt s/a")
            return sbmax,c1,snowmmax,snowmaxdict,"snowball technique and parallel investment"
        else:
            print("avalanche is better doing debts with debt s/a")
            return avmax,a1,avmmax,avmaxdixt ,"avalanche technique and parallel investment"       
    else:
        for i in sb1:
            if i[1]>sbmax:
                sbmax=i[1]
                snowmmax=i[2]
                snowmaxdict=i[3]
                c1=i[4]
        for i in av1:
            if i[1]>avmax:
                avmax=i[1]
                avmmax=i[2]
                avmaxdixt=i[3]
                a1=i[4]
        if sbmax>=avmax:
            print("snow ball is better for parallel debt s/a and investment")
            return sbmax,c1,snowmmax,snowmaxdict,"snowball technique"
        else:
            print("avalanche is better parallel debt s/a and investment")
            return avmax,a1,avmmax,avmaxdixt,"avalanche technique"
    
def debtsaandnoparallelinvest(snb,avl,duration,monthlyleftover):
    sb=[]
    csb=[]
    cal=[]
    sb1=[]
    for i in range(0,10):
        i=0.1*i
        csb=[]
        csb1=[]
        cashflow_sb=0
        cashflow_sb1=0
        #cashflow_sb1=0
        sbdict,sbmdict=snowball(snb,i*monthlyleftover)
        for key,value in sbmdict.items():
            if type(value) is dict:
                 if value['month']==sbdict['months']:
                     cashflow_sb+=value['monthlybalance'][-2]+value['interest'][-1]
        
        diff=monthlyleftover-(i*monthlyleftover)
        ctemp1=0
        cashflow_sb1+=diff*duration
        if duration<sbdict['months']:
            for i in range(duration):
                 ctemp1+=diff
                 csb1.append(ctemp1)
            sb1.append([i*monthlyleftover,cashflow_sb1,sbmdict,sbdict,csb1]) 
        else:
            ctemp1=0
            cashflow_sb+=diff*(sbdict['months'])
            for i in range(sbdict['months']-1):
                ctemp1+=diff
                csb.append(ctemp1)
            cashflow_sb=(monthlyleftover-cashflow_sb)
            csb.append(cashflow_sb)
            ctemp1=cashflow_sb
            cashflow_sb+=monthlyleftover*(duration-sbdict['months'])
            for i in range(duration-sbdict['months']):
                ctemp1+=monthlyleftover
                csb.append(ctemp1)
            sb.append([i*monthlyleftover,cashflow_sb,sbmdict,sbdict,csb])   
    av=[]
    av1=[]
    for i in range(0,10): 
        i=0.1*i
        cal=[]
        cal1=[]
        cashflow_av=0
        cashflow_av1=0
        #cashflow_av1=0
        avdict,avldict=avalanche(avl,i*monthlyleftover)     
        for key,value in avldict.items():
            if type(value) is dict:
                 if value['month']==avdict['months']:
                     cashflow_av+=value['monthlybalance'][-2]+value['interest'][-1]
        diff=monthlyleftover-(i*monthlyleftover)
        ctemp2=0
        cashflow_av1+=diff*duration
        if duration<sbdict['months']:
            for i in range(duration):
                 ctemp2+=diff
                 cal1.append(ctemp2)
            av1.append([i*monthlyleftover,cashflow_av1,avldict,avdict,cal1]) 
        else:
            for i in range(avdict['months']-1):
                ctemp2+=diff
                cal.append(ctemp2)
            cashflow_av+=diff*avdict['months']
            cashflow_av+=(monthlyleftover-cashflow_av)
            csb.append(cashflow_av)
            cashflow_av+=monthlyleftover*(duration-avdict['months'])
            for i in range(duration-avdict['months']):
                ctemp2+=monthlyleftover
                cal.append(ctemp2)
            av.append([i*monthlyleftover,cashflow_av,avldict,avdict,cal])  
    sbmax=0
    avmax=0
    c1=[]
    a1=[]
    snowmaxdict={}
    avmaxdixt={}
    if sb==[] or av==[]:
        for i in sb1:
            if i[1]>sbmax:
                sbmax=i[1]
                snowmmax=i[2]
                snowmaxdict=i[3]
                c1=i[4]
        for i in av1:
            if i[1]>avmax:
                avmax=i[1]
                avmmax=i[2]
                avmaxdixt=i[3]
                a1=i[4]
        if sbmax>=avmax:
            print("snow ball is better for doing debts with debt s/a")
            return sbmax,c1,snowmmax,snowmaxdict,"snowball technique"
        else:
            print("avalanche is better doing debts with debt s/a")
            return avmax,a1,avmmax,avmaxdixt ,"avalanche technique"    
    for i in sb:
        if i[1]>sbmax:
            sbmax=i[1]
            snowmmax=i[2]
            snowmaxdict=i[3]
            c1=i[4]
    for i in av:
        if i[1]>avmax:
            avmax=i[1]
            avmmax=i[2]
            avmaxdixt=i[3]
            a1=i[4]
    if sbmax>=avmax:
        print("snow ball is better for parallel debt s/a and investment")
        return sbmax,c1,snowmmax,snowmaxdict,"snowball technique"
    else:
        print("avalanche is better for parallel debt s/a and investment")
        return avmax,a1,avmmax,avmaxdixt,"avalanche technique"
    
def debtnormandnoinvest(normal_out,normal_msplit,duration,monthlyleftover):
    _max=0
    cnm=[]
    cnm1=[]
    cashflow1=0
    monthlyleftover_temp=monthlyleftover
    for key,value in normal_msplit.items():
             if type(value) is dict:
                 if _max<=len(value['month'])-1:
                     _max=len(value['month'])-1
    i=0
    clm=[]
    while i<=_max:
        monthlyleftover_temp=monthlyleftover
        for key,value in normal_msplit.items():
             if type(value) is dict:
                 if i<len(value['month']):                    
                     if value['monthlybalance'][i]>=value['minimumdue']:
                         monthlyleftover_temp-=value['minimumdue']
                     elif value['monthlybalance'][i]==0.0 :
                         monthlyleftover_temp-=value['monthlybalance'][i-1]+value['interest'][i]
        #print(monthlyleftover_temp)
        i+=1
        clm.append(monthlyleftover_temp)
    #print(clm)
    if duration<normal_out['months']:
        clm=clm[:duration]
        for i in clm:
            cashflow1+=i
            cnm1.append(cashflow1)
        return cashflow1,normal_out,cnm1
        
    cashflow=0
    for i in clm:
        cashflow+=i
        cnm.append(cashflow)
    ctemp=cashflow
    cashflow+=monthlyleftover*(duration-_max)
    for i in range(duration-_max-1):
            ctemp+=monthlyleftover
            cnm.append(ctemp)    
    return cashflow,normal_out,cnm
        
@app.route("/saveinterestdownload", methods=['GET','POST'])
def saveinterestdownload():  
    data = request.get_json()
    print("data")
    print(data)
    debts=data['debts']
    extraMoney=float(data['extraMoney'])
    debtname=[]
    minimumdue=[]
    balance=[]
    interestrate=[]
    
    for debt in  debts:
        debtname.append(debt['account_type'])
        balance.append(float(debt['balance']))
        minimumdue.append(float(debt['minimum_due']))
        interestrate.append(float(debt['interest']))
    """
    debtname=['debt1','debt2','debt3','debt4']
    extraMoney=500
    minimumdue=[100,200,300,400]
    #monthlyleftover=sum(minimumdue)+extraMoney
    interestrate=[1,2,3,4]
    balance=[1000,2000,3000,5000]
    """
    months=[0,0,0,0]
    totalPaid=[0,0,0,0]
    totalInterestPaid=[0,0,0,0]
    #count=0
    debtslist=[]
    #cashflow=100000
    #duration=20*12
    #rateofinterest=5
    ##Add debts to  nested list
    for i in range(len(balance)):
        debtslist.append([debtname[i],minimumdue[i],interestrate[i],balance[i],totalPaid[i],months[i],totalInterestPaid[i]])
        ##Call snowball and avalanche and normal  function
    normal_out,normal_msplit=normal(debtslist,minimumdue)
    debtslist2=debtslist.copy()
    snowball_out,snowball_msplit=snowball(debtslist2,extraMoney)
    debtslist3=debtslist.copy()
    avalanche_out,avalanche_msplit=avalanche(debtslist3,extraMoney)

    message,best_out=savinginterest(snowball_out,avalanche_out)
    if best_out==snowball_out:
       df= converttocsv(snowball_msplit)
    else:
       df= converttocsv(avalanche_msplit)
    resp = make_response(df.to_csv())
    resp.headers["Content-Disposition"] = "attachment; filename=saveinterest.csv"
    resp.headers["Content-Type"] = "text/csv"
    return resp
    
@app.route("/saveinterest", methods=['GET','POST'])

def saveinterest():
    data = request.get_json()
    print("data")
    print(data)
    debts=data['debts']
    extraMoney=float(data['extraMoney'])
    debtname=[]
    minimumdue=[]
    balance=[]
    interestrate=[]
    
    for debt in  debts:
        debtname.append(debt['account_type'])
        balance.append(float(debt['balance']))
        minimumdue.append(float(debt['minimum_due']))
        interestrate.append(float(debt['interest']))
        
    #monthlyleftover=sum(minimumdue)+extraMoney    
    """
    debtname=['debt1','debt2','debt3','debt4']
    extraMoney=500
    minimumdue=[100,200,300,400] 
    monthlyleftover=sum(minimumdue)+extraMoney
    balance=[1000,2000,3000,5000]
    interestrate=[1,2,3,4]
    """
    
    months=[0,0,0,0]
    totalPaid=[0,0,0,0]
    totalInterestPaid=[0,0,0,0]
    #count=0
    debtslist=[]
    #cashflow=100000
    #duration=20*12
    #rateofinterest=5
    ##Add debts to  nested list
    for i in range(len(balance)):
        debtslist.append([debtname[i],minimumdue[i],interestrate[i],balance[i],totalPaid[i],months[i],totalInterestPaid[i]])
        ##Call snowball and avalanche and normal  function
    normal_out,normal_msplit=normal(debtslist,minimumdue)
    debtslist2=debtslist.copy()
    snowball_out,snowball_msplit=snowball(debtslist2,extraMoney)
    debtslist3=debtslist.copy()
    avalanche_out,avalanche_msplit=avalanche(debtslist3,extraMoney)
    
    print("monthly splitup snowball:")
    print(snowball_msplit)
    print("--------------------------")
    print("monthly splitup avalanche:")
    print(avalanche_msplit)
    print("--------------------------")
    print("monthly splitup normal:")
    print(normal_msplit)
    print("--------------------------")
    print("Debt list snowball dictionary:")
    print(snowball_out)
    print("--------------------------")
    print("Debt list avalanche dictionary:")
    print(avalanche_out)
    print("--------------------------")
    print("Debt list normal dictionary:")
    print(normal_out)
    print("--------------------------")
    
    message,best_out=savinginterest(snowball_out,avalanche_out)
    best=[]
    for key,value in best_out.items():
        if type(value) is dict:
            best.append(value)
    print("best")
    print(best)   
    return jsonify({"Optimalmethod":message,"saveinterest":best,"totalpaid":best_out['totalpaid'],
    "totalmonths":best_out['months'],"normalpaid":normal_out['totalpaid'],
    "normalmonths":normal_out['months'],"paiddiff":(normal_out['totalpaid']-best_out['totalpaid']),
    "monthdiff":(normal_out['months']-best_out['months'])})

@app.route("/feasibilityinloansdownload", methods=['GET','POST'])

def feasibilityinloansdownload():
    data = request.get_json()
    print("data")
    print(data)
    debts=data['debts']
    extraMoney=float(data['extraMoney'])
    debtname=[]
    minimumdue=[]
    balance=[]
    interestrate=[]
    
    for debt in  debts:
        debtname.append(debt['account_type'])
        balance.append(float(debt['balance']))
        minimumdue.append(float(debt['minimum_due']))
        interestrate.append(float(debt['interest']))
    """
    debtname=['debt1','debt2','debt3','debt4']
    extraMoney=500
    minimumdue=[100,200,300,400]
    #monthlyleftover=sum(minimumdue)+extraMoney
    interestrate=[1,2,3,4]
    balance=[1000,2000,3000,5000]
    """
    months=[0,0,0,0]
    totalPaid=[0,0,0,0]
    totalInterestPaid=[0,0,0,0]
    #count=0
    debtslist=[]
    #cashflow=100000
    #duration=20*12
    #rateofinterest=5
    ##Add debts to  nested list
    for i in range(len(balance)):
        debtslist.append([debtname[i],minimumdue[i],interestrate[i],balance[i],totalPaid[i],months[i],totalInterestPaid[i]])
        ##Call snowball and avalanche and normal  function
    normal_out,normal_msplit=normal(debtslist,minimumdue)
    debtslist2=debtslist.copy()
    snowball_out,snowball_msplit=snowball(debtslist2,extraMoney)
    debtslist3=debtslist.copy()
    avalanche_out,avalanche_msplit=avalanche(debtslist3,extraMoney)
 
    message,best_out=feasiblityinloans(snowball_out,avalanche_out)
    if best_out==snowball_out:
       df= converttocsv(snowball_msplit)
    else:
       df= converttocsv(avalanche_msplit)
    resp = make_response(df.to_csv())
    resp.headers["Content-Disposition"] = "attachment; filename=feasbilityinloans.csv"
    resp.headers["Content-Type"] = "text/csv"
    return resp
@app.route("/feasibilityinloans", methods=['GET','POST'])

def feasibilityinloans():
    data = request.get_json()
    print("data")
    print(data)
    debts=data['debts']
    extraMoney=float(data['extraMoney'])
    debtname=[]
    minimumdue=[]
    balance=[]
    interestrate=[]
    
    for debt in  debts:
        debtname.append(debt['account_type'])
        balance.append(float(debt['balance']))
        minimumdue.append(float(debt['minimum_due']))
        interestrate.append(float(debt['interest']))
    """
    debtname=['debt1','debt2','debt3','debt4']
    extraMoney=500
    minimumdue=[100,200,300,400]
    #monthlyleftover=sum(minimumdue)+extraMoney
    interestrate=[1,2,3,4]
    balance=[1000,2000,3000,5000]
    """
    months=[0,0,0,0]
    totalPaid=[0,0,0,0]
    totalInterestPaid=[0,0,0,0]
    #count=0
    debtslist=[]
    #cashflow=100000
    #duration=20*12
    #rateofinterest=5
    ##Add debts to  nested list
    for i in range(len(balance)):
        debtslist.append([debtname[i],minimumdue[i],interestrate[i],balance[i],totalPaid[i],months[i],totalInterestPaid[i]])
        ##Call snowball and avalanche and normal  function
    normal_out,normal_msplit=normal(debtslist,minimumdue)
    debtslist2=debtslist.copy()
    snowball_out,snowball_msplit=snowball(debtslist2,extraMoney)
    debtslist3=debtslist.copy()
    avalanche_out,avalanche_msplit=avalanche(debtslist3,extraMoney)
    print("monthly splitup snowball:")
    print(snowball_msplit)
    print("--------------------------")
    print("monthly splitup avalanche:")
    print(avalanche_msplit)
    print("--------------------------")
    print("monthly splitup normal:")
    print(normal_msplit)
    print("--------------------------")
    print("Debt list snowball dictionary:")
    print(snowball_out)
    print("--------------------------")
    print("Debt list avalanche dictionary:")
    print(avalanche_out)
    print("--------------------------")
    print("Debt list normal dictionary:")
    print(normal_out)
    print("--------------------------")
    
    message,best_out=feasiblityinloans(snowball_out,avalanche_out)
    best=[]
    for key,value in best_out.items():
        if type(value) is dict:
            best.append(value)
    print("best")
    print(best)   
    return jsonify({"Optimalmethod":message,"feasibilityinloans":best,"totalpaid":best_out['totalpaid'],
    "totalmonths":best_out['months'],"normalpaid":normal_out['totalpaid'],
    "paiddiff":(normal_out['totalpaid']-best_out['totalpaid']),
    "normalmonths":normal_out['months'],"monthdiff":(normal_out['months']-best_out['months'])})

@app.route("/increasecashflowwithinvestmentdownload", methods=['GET','POST'])
def increasecashflowwithinvestmentdownload():
    """
    debtname=['debt1','debt2','debt3','debt4']
    extraMoney=500
    minimumdue=[100,200,300,400]
    monthlyleftover=sum(minimumdue)+extraMoney
    interestrate=[1,2,3,4]
    balance=[1000,2000,3000,5000]
    months=[0,0,0,0]
    totalPaid=[0,0,0,0]
    totalInterestPaid=[0,0,0,0]
    debtslist=[]
    duration=20*12
    rateofinterest=5
    """
    data = request.get_json()
    print("data")
    print(data)
    debts=data['debts']
    extraMoney=float(data['extraMoney'])
    debtname=[]
    minimumdue=[]
    balance=[]
    interestrate=[]
    months=[0,0,0,0]
    totalPaid=[0,0,0,0]
    totalInterestPaid=[0,0,0,0]
    duration=int(data['year'])*12
    rateofinterest=float(data['roi'])
    #count=0
    debtslist=[]
    for debt in  debts:
        debtname.append(debt['account_type'])
        balance.append(float(debt['balance']))
        minimumdue.append(float(debt['minimum_due']))
        interestrate.append(float(debt['interest']))
        
    monthlyleftover=sum(minimumdue)+extraMoney
    ##Add debts to  nested list
    for i in range(len(balance)):
        debtslist.append([debtname[i],minimumdue[i],interestrate[i],balance[i],totalPaid[i],months[i],totalInterestPaid[i]])
        ##Call snowball and avalanche and normal  function
    normal_out,normal_msplit=normal(debtslist,minimumdue)
    debtslist2=debtslist.copy()
    snowball_out,snowball_msplit=snowball(debtslist2,extraMoney)
    debtslist3=debtslist.copy()
    avalanche_out,avalanche_msplit=avalanche(debtslist3,extraMoney)
    print("monthly splitup snowball:")
    print(snowball_msplit)
    print("--------------------------")
    print("monthly splitup avalanche:")
    print(avalanche_msplit)
    print("--------------------------")
    print("monthly splitup normal:")
    print(normal_msplit)
    print("--------------------------")
    print("Debt list snowball dictionary:")
    print(snowball_out)
    print("--------------------------")
    print("Debt list avalanche dictionary:")
    print(avalanche_out)
    print("--------------------------")
    print("Debt list normal dictionary:")
    print(normal_out)
    print("--------------------------")
  
    ##Cash flow
    ##Debt free and then investment
    ## Case 1
    cash1,best_out1,cf1,message1=debtfreeandinvestment(snowball_out,avalanche_out,snowball_msplit,avalanche_msplit,rateofinterest,duration,monthlyleftover,debtslist2,debtslist3)
    print("Cashflow for debtfree and then investment is :")
    print(cash1)
    if best_out1==snowball_out:
        msplit1=snowball_msplit
    else:
        msplit1=avalanche_msplit
    ##Case 2
    ##Debt minimum due and parallel investment
    cash2,best_out2,cf2,message2=debtminandparallelinvest(normal_out,normal_msplit,rateofinterest,duration,monthlyleftover)
    print("Cashflow for debtmin due and parallel investment is:")
    print(cash2)
    msplit2=normal_msplit
    ##Case 3
    ##snowball and avalanche at parallel
    
    ##Case 3
    ##Debt snowball and avalanche and parallel investment
    cash3,cf3,msplit3,best_out3,message3=debtsaandparallelinvest(debtslist2,debtslist3,rateofinterest,duration,monthlyleftover)
    print("Cashflow for debt snow/aval due and parallel investment is:")
    print(cash3)
    bestlist=[[cash1,"Cashflow for debtfree and then investment is better",cf1,best_out1,msplit1],[cash2,"Cashflow for debtmin due and parallel investment is better",cf2,best_out2,msplit2],
              [cash3,"Cashflow for debtmin due and parallel investment is better",cf3,best_out3,msplit3]]
    ch=0
    cf=[]
    
    for i in bestlist:
        if i[0]>ch:
            ch=i[0] 
            bm=i[4]
            cf=i[2]
    df1= converttocsv(bm)
    df2=pd.DataFrame(cf[:duration],columns=['cashflow'])
    df=pd.concat([df1,df2],axis=1)
    resp = make_response(df.to_csv())
    resp.headers["Content-Disposition"] = "attachment; filename=investmentcasfflow.csv"
    resp.headers["Content-Type"] = "text/csv"
    return resp


@app.route("/increasecashflowwithinvestment", methods=['GET','POST'])

def increasecashflowwithinvestment():
    """
    debtname=['debt1','debt2','debt3','debt4']
    extraMoney=500
    minimumdue=[100,200,300,400]
    monthlyleftover=sum(minimumdue)+extraMoney
    interestrate=[1,2,3,4]
    balance=[1000,2000,3000,5000]
    months=[0,0,0,0]
    totalPaid=[0,0,0,0]
    totalInterestPaid=[0,0,0,0]
    debtslist=[]
    duration=20*12
    rateofinterest=5
    """
    data = request.get_json()
    print("data")
    print(data)
    debts=data['debts']
    extraMoney=float(data['extraMoney'])
    debtname=[]
    minimumdue=[]
    balance=[]
    interestrate=[]
    months=[0,0,0,0]
    totalPaid=[0,0,0,0]
    totalInterestPaid=[0,0,0,0]
    duration=int(data['year'])*12
    rateofinterest=float(data['roi'])
    #count=0
    debtslist=[]
    for debt in  debts:
        debtname.append(debt['account_type'])
        balance.append(float(debt['balance']))
        minimumdue.append(float(debt['minimum_due']))
        interestrate.append(float(debt['interest']))
        
    monthlyleftover=sum(minimumdue)+extraMoney
    ##Add debts to  nested list
    for i in range(len(balance)):
        debtslist.append([debtname[i],minimumdue[i],interestrate[i],balance[i],totalPaid[i],months[i],totalInterestPaid[i]])
        ##Call snowball and avalanche and normal  function
    normal_out,normal_msplit=normal(debtslist,minimumdue)
    debtslist2=debtslist.copy()
    snowball_out,snowball_msplit=snowball(debtslist2,extraMoney)
    debtslist3=debtslist.copy()
    avalanche_out,avalanche_msplit=avalanche(debtslist3,extraMoney)
  
    ##Cash flow
    ##Debt free and then investment
    ## Case 1
    cash1,best_out1,cf1,message1=debtfreeandinvestment(snowball_out,avalanche_out,snowball_msplit,avalanche_msplit,rateofinterest,duration,monthlyleftover,debtslist2,debtslist3)
    print("Cashflow for debtfree and then investment is :")
    print(cash1)
    
    ##Case 2
    ##Debt minimum due and parallel investment
    cash2,best_out2,cf2,message2=debtminandparallelinvest(normal_out,normal_msplit,rateofinterest,duration,monthlyleftover)
    print("Cashflow for debtmin due and parallel investment is:")
    print(cash2)
    
    ##Case 3
    ##snowball and avalanche at parallel
    
    ##Case 3
    ##Debt snowball and avalanche and parallel investment
    cash3,cf3,msplit,best_out3,message3=debtsaandparallelinvest(debtslist2,debtslist3,rateofinterest,duration,monthlyleftover)
    print("Cashflow for debt snow/aval due and parallel investment is:")
    print(cash3)
    bestlist=[[cash1,message1,cf1,best_out1],[cash2,message2,cf2,best_out2],
              [cash3,message3,cf3,best_out3]]
    ch=0
    message=""
    cf=[]
    best={}
    for i in bestlist:
        if i[0]>ch:
            ch=i[0] 
            message=i[1]
            cf=i[2]
            best=i[3]
    best1=[]
    for key,value in best.items():
        if type(value) is dict:
            best1.append(value)
    print("best")
    print(best1)  
    return jsonify({"Optimalmethod":message,"cashflow":ch,"optimallist":cf[:duration],"debt":best1,"cashflowdiff":ch-cash2,"normalcashflow":cash2})

@app.route("/increasecashflowwithoutinvestmentdownload", methods=['GET','POST'])

def increasecashflowwithoutinvestmentdownload():
    """
    debtname=['debt1','debt2','debt3','debt4']
    extraMoney=500
    minimumdue=[100,200,300,400]
    monthlyleftover=sum(minimumdue)+extraMoney
    interestrate=[1,2,3,4]
    balance=[1000,2000,3000,5000]
    months=[0,0,0,0]
    totalPaid=[0,0,0,0]
    totalInterestPaid=[0,0,0,0]
    debtslist=[]
    duration=20*12
    """
    data = request.get_json()
    print("data")
    print(data)
    debts=data['debts']
    extraMoney=float(data['extraMoney'])
    debtname=[]
    minimumdue=[]
    balance=[]
    interestrate=[]
    months=[0,0,0,0]
    totalPaid=[0,0,0,0]
    totalInterestPaid=[0,0,0,0]
    duration=int(data['year'])*12
    #count=0
    debtslist=[]
    for debt in  debts:
        debtname.append(debt['account_type'])
        balance.append(float(debt['balance']))
        minimumdue.append(float(debt['minimum_due']))
        interestrate.append(float(debt['interest']))
        
    monthlyleftover=sum(minimumdue)+extraMoney
    ##Add debts to  nested list
    for i in range(len(balance)):
        debtslist.append([debtname[i],minimumdue[i],interestrate[i],balance[i],totalPaid[i],months[i],totalInterestPaid[i]])
        ##Call snowball and avalanche and normal  function
    normal_out,normal_msplit=normal(debtslist,minimumdue)
    debtslist2=debtslist.copy()
    snowball_out,snowball_msplit=snowball(debtslist2,extraMoney)
    debtslist3=debtslist.copy()
    avalanche_out,avalanche_msplit=avalanche(debtslist3,extraMoney)
    print("monthly splitup snowball:")
    print(snowball_msplit)
    print("--------------------------")
    print("monthly splitup avalanche:")
    print(avalanche_msplit)
    print("--------------------------")
    print("monthly splitup normal:")
    print(normal_msplit)
    print("--------------------------")
    print("Debt list snowball dictionary:")
    print(snowball_out)
    print("--------------------------")
    print("Debt list avalanche dictionary:")
    print(avalanche_out)
    print("--------------------------")
    print("Debt list normal dictionary:")
    print(normal_out)
    print("--------------------------")

    ##Debt snowball and avalanche and no parallel investment
    cash,cf,msplit,best_out,message=debtsaandnoparallelinvest(debtslist2,debtslist3,duration,monthlyleftover)
    print("Cashflow for debt snow/aval due and  no parallel investment is:")
    print(cash)
    df1= converttocsv(msplit)
    df2=pd.DataFrame(cf[:duration],columns=['cashflow'])
    df=pd.concat([df1,df2],axis=1)
    resp = make_response(df.to_csv())
    resp.headers["Content-Disposition"] = "attachment; filename=investmentcasfflownoinvestment.csv"
    resp.headers["Content-Type"] = "text/csv"
    return resp
 

@app.route("/increasecashflowwithoutinvestment", methods=['GET','POST'])

def increasecashflowwithoutinvestment():
    """
    debtname=['debt1','debt2','debt3','debt4']
    extraMoney=500
    minimumdue=[100,200,300,400]
    
    interestrate=[1,2,3,4]
    balance=[1000,2000,3000,5000]
    months=[0,0,0,0]
    totalPaid=[0,0,0,0]
    totalInterestPaid=[0,0,0,0]
    debtslist=[]
    duration=20*12
    """
    data = request.get_json()
    print("data")
    print(data)
    debts=data['debts']
    extraMoney=float(data['extraMoney'])
    debtname=[]
    minimumdue=[]
    balance=[]
    interestrate=[]
    months=[0,0,0,0]
    totalPaid=[0,0,0,0]
    totalInterestPaid=[0,0,0,0]
    duration=int(data['year'])*12
    #count=0
    debtslist=[]
    for debt in  debts:
        debtname.append(debt['account_type'])
        balance.append(float(debt['balance']))
        minimumdue.append(float(debt['minimum_due']))
        interestrate.append(float(debt['interest']))
        
    monthlyleftover=sum(minimumdue)+extraMoney
    ##Add debts to  nested list
    for i in range(len(balance)):
        debtslist.append([debtname[i],minimumdue[i],interestrate[i],balance[i],totalPaid[i],months[i],totalInterestPaid[i]])
        ##Call snowball and avalanche and normal  function
    normal_out,normal_msplit=normal(debtslist,minimumdue)
    debtslist2=debtslist.copy()
    snowball_out,snowball_msplit=snowball(debtslist2,extraMoney)
    debtslist3=debtslist.copy()
    avalanche_out,avalanche_msplit=avalanche(debtslist3,extraMoney)
    print("monthly splitup snowball:")
    print(snowball_msplit)
    print("--------------------------")
    print("monthly splitup avalanche:")
    print(avalanche_msplit)
    print("--------------------------")
    print("monthly splitup normal:")
    print(normal_msplit)
    print("--------------------------")
    print("Debt list snowball dictionary:")
    print(snowball_out)
    print("--------------------------")
    print("Debt list avalanche dictionary:")
    print(avalanche_out)
    print("--------------------------")
    print("Debt list normal dictionary:")
    print(normal_out)
    print("--------------------------")

    ##Debt snowball and avalanche and no parallel investment
    cash,cf,msplit,best_out,message=debtsaandnoparallelinvest(debtslist2,debtslist3,duration,monthlyleftover)
    
    print("Cashflow for debt snow/aval due and  no parallel investment is:")
    print(cash)
    best=[]
    for key,value in best_out.items():
        if type(value) is dict:
            best.append(value)
    print("best")
    print(best)  
    ncash,ncf,ncashsplit=debtnormandnoinvest(normal_out,normal_msplit,duration,monthlyleftover)
    return jsonify({"cashflow":cash,"optimallist":cf,"debt":best,"Optimalmethod":message,"normalcashflow":ncash,"cashflowdiff":cash-ncash})
if __name__ == '__main__':
    app.run(debug=False,host='0.0.0.0',port=5000)